In [ ]:
# Imports and setup
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure output directories exist
os.makedirs('/home/jovyan/work/outputs', exist_ok=True)
os.makedirs('/home/jovyan/work/outputs/maps', exist_ok=True)
os.makedirs('/home/jovyan/work/outputs/figures', exist_ok=True)

print('Environment ready.')


In [ ]:
# Load datasets
energy_path = '/home/jovyan/work/datasets/energyreporter_municipality_historized.csv'
wealth_path = '/home/jovyan/work/datasets/data_7354970.csv'
geojson_path = '/home/jovyan/work/datasets/zh_municipalities.geojson'

energy = pd.read_csv(energy_path, parse_dates=['energyreporter_date'])
wealth = pd.read_csv(wealth_path, sep=';')

geo = None
if os.path.exists(geojson_path):
    try:
        geo = gpd.read_file(geojson_path)
    except Exception as e:
        print(f'Warning: Failed to load GeoJSON: {e}')
else:
    print('Info: GeoJSON not found; maps will be skipped until provided.')

print(energy.shape, wealth.shape, 'geo loaded' if geo is not None else 'no geo')


In [ ]:
# Filter to Zurich and essential columns
energy_zh = energy.loc[energy['canton'] == 'ZH', ['bfs_nr', 'energyreporter_date', 'elec_consumption_mwh_per_year_per_capita']].copy()
# normalize types
energy_zh['bfs_nr'] = energy_zh['bfs_nr'].astype(int)

wealth_small = wealth.loc[:, ['BFS_NR', 'INDIKATOR_ID', 'INDIKATOR_NAME', 'INDIKATOR_JAHR', 'INDIKATOR_VALUE']].copy()
wealth_small['BFS_NR'] = wealth_small['BFS_NR'].astype(int)

if geo is not None:
    # try common BFS field names
    for cand in ['BFS', 'BFS_NUMMER', 'BFS_NUM', 'GEMEINDENO', 'GMDNR', 'bfs_nr']:
        if cand in geo.columns:
            geo = geo.rename(columns={cand: 'bfs_nr'})
            break
    geo['bfs_nr'] = geo['bfs_nr'].astype(int)

print(energy_zh.shape, wealth_small.shape)


In [ ]:
# Aggregate energy 2024 monthly → yearly average per capita
energy_2024 = energy_zh[energy_zh['energyreporter_date'].dt.year == 2024].copy()
consum = (energy_2024
          .groupby('bfs_nr', as_index=False)['elec_consumption_mwh_per_year_per_capita']
          .mean()
          .rename(columns={'elec_consumption_mwh_per_year_per_capita': 'consum_per_capita_mwh_2024'}))

consum.head()


In [ ]:
# Wealth 2022: select median/avg/total-per-capita for income and capital, pivot wide
wealth_2022 = wealth_small[wealth_small['INDIKATOR_JAHR'] == 2022].copy()
patterns = {
    'income_median_2022': r'^Steuerb\. Einkommen.*Median',
    'income_avg_2022': r'^Steuerb\. Einkommen.*Durchschn',
    'income_total_per_capita_2022': r'^Steuerb\. Einkommen.*(je Einwohner|pro Kopf)',
    'capital_median_2022': r'^Steuerb\. Vermögen.*Median',
    'capital_avg_2022': r'^Steuerb\. Vermögen.*Durchschn',
    'capital_total_per_capita_2022': r'^Steuerb\. Vermögen.*(je Einwohner|pro Kopf)'
}
wealth_2022['metric'] = np.nan
for col, pat in patterns.items():
    m = wealth_2022['INDIKATOR_NAME'].str.contains(pat, regex=True, na=False)
    wealth_2022.loc[m, 'metric'] = col

wide = (wealth_2022.dropna(subset=['metric'])
        .pivot_table(index='BFS_NR', columns='metric', values='INDIKATOR_VALUE', aggfunc='first')
        .reset_index()
        .rename(columns={'BFS_NR': 'bfs_nr'}))

wide.head()


In [ ]:
# Join and derive ratios
merged = consum.merge(wide, on='bfs_nr', how='inner')

# Replace 0 to avoid div by zero
for col in [
    'income_median_2022','income_avg_2022','income_total_per_capita_2022',
    'capital_median_2022','capital_avg_2022','capital_total_per_capita_2022'
]:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors='coerce')
        merged[col] = merged[col].replace(0, np.nan)

# Ratios
if 'capital_median_2022' in merged.columns:
    merged['ratio_consum_to_capital_median'] = merged['consum_per_capita_mwh_2024'] / merged['capital_median_2022']
if 'capital_avg_2022' in merged.columns:
    merged['ratio_consum_to_capital_avg'] = merged['consum_per_capita_mwh_2024'] / merged['capital_avg_2022']
if 'capital_total_per_capita_2022' in merged.columns:
    merged['ratio_consum_to_capital_totalpc'] = merged['consum_per_capita_mwh_2024'] / merged['capital_total_per_capita_2022']
if 'income_median_2022' in merged.columns:
    merged['ratio_consum_to_income_median'] = merged['consum_per_capita_mwh_2024'] / merged['income_median_2022']
if 'income_avg_2022' in merged.columns:
    merged['ratio_consum_to_income_avg'] = merged['consum_per_capita_mwh_2024'] / merged['income_avg_2022']
if 'income_total_per_capita_2022' in merged.columns:
    merged['ratio_consum_to_income_totalpc'] = merged['consum_per_capita_mwh_2024'] / merged['income_total_per_capita_2022']

# Save merged data
merged.to_csv('/home/jovyan/work/outputs/merged_zh_wealth_energy.csv', index=False)
merged.head()


In [ ]:
# Choropleth maps (if geo available)
if geo is not None:
    geo_merged = geo.merge(merged, on='bfs_nr', how='left')
    vars_to_map = [
        'consum_per_capita_mwh_2024',
        'capital_median_2022',
        'income_median_2022',
        'ratio_consum_to_capital_median'
    ]
    for var in vars_to_map:
        if var in geo_merged.columns:
            ax = geo_merged.plot(column=var, scheme='Quantiles', k=5, legend=True, figsize=(8,8),
                                 cmap='viridis', missing_kwds={'color': 'lightgrey', 'hatch': '///', 'label': 'No data'})
            ax.set_axis_off()
            ax.set_title(var)
            plt.tight_layout()
            outpath = f'/home/jovyan/work/outputs/maps/{var}.png'
            plt.savefig(outpath, dpi=150)
            plt.close()
else:
    print('GeoJSON not available; skipping maps.')


In [ ]:
# Correlations and scatter with regression
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression

corr_rows = []
wealth_cols = [c for c in [
    'income_median_2022','income_avg_2022','income_total_per_capita_2022',
    'capital_median_2022','capital_avg_2022','capital_total_per_capita_2022'
] if c in merged.columns]

for col in wealth_cols:
    df = merged[['consum_per_capita_mwh_2024', col]].dropna()
    if df.empty:
        continue
    pr, pp = pearsonr(df['consum_per_capita_mwh_2024'], df[col])
    sr, sp = spearmanr(df['consum_per_capita_mwh_2024'], df[col])
    corr_rows.append({'metric': col, 'pearson_r': pr, 'pearson_p': pp, 'spearman_r': sr, 'spearman_p': sp, 'n': len(df)})

corr_df = pd.DataFrame(corr_rows).sort_values('pearson_r', ascending=False)
corr_df.to_csv('/home/jovyan/work/outputs/correlations.csv', index=False)
corr_df


In [ ]:
# Scatterplots with regression lines
sns.set_context('notebook')
for col in wealth_cols:
    if col not in merged.columns:
        continue
    df = merged[['consum_per_capita_mwh_2024', col]].dropna()
    if df.empty:
        continue
    plt.figure(figsize=(6,5))
    sns.regplot(data=df, x='consum_per_capita_mwh_2024', y=col, scatter_kws={'alpha':0.6})
    plt.title(f'Energy per capita vs {col}')
    plt.xlabel('Energy per capita (MWh) 2024')
    plt.ylabel(col)
    plt.tight_layout()
    plt.savefig(f'/home/jovyan/work/outputs/figures/scatter_{col}.png', dpi=150)
    plt.close()
print('Saved scatter plots.')


In [ ]:
# Outlier detection via linear regression residuals
from sklearn.preprocessing import StandardScaler

outlier_rows = []
for col in wealth_cols:
    df = merged[['bfs_nr','consum_per_capita_mwh_2024', col]].dropna()
    if df.shape[0] < 10:
        continue
    X = df[[col]].values.reshape(-1,1)
    y = df['consum_per_capita_mwh_2024'].values
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    resid = y - y_pred
    z = (resid - resid.mean()) / (resid.std(ddof=1) if resid.std(ddof=1) else 1.0)
    df_out = pd.DataFrame({'bfs_nr': df['bfs_nr'].values, 'metric': col, 'residual': resid, 'zscore': z})
    outlier_rows.append(df_out)

outliers = pd.concat(outlier_rows, ignore_index=True) if outlier_rows else pd.DataFrame(columns=['bfs_nr','metric','residual','zscore'])
outliers_sorted = outliers.reindex(outliers['zscore'].abs().sort_values(ascending=False).index)
outliers.to_csv('/home/jovyan/work/outputs/outliers.csv', index=False)
outliers_sorted.head(10)
